# Task 1

In [2]:
import pandas as pd
import gensim
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import random

In [36]:
#Assining URL to variable to get the dataset.

url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"

# Loading dataset in dataframe

df = pd.read_csv(url, sep='\t', compression='gzip', error_bad_lines=False, dtype='str')

<ipython-input-36-6b4191d545d7>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(url, sep='\t', compression='gzip', error_bad_lines=False, dtype='str')
Skipping line 20773: expected 15 fields, saw 22
Skipping line 39834: expected 15 fields, saw 22
Skipping line 52957: expected 15 fields, saw 22
Skipping line 54540: expected 15 fields, saw 22

Skipping line 80276: expected 15 fields, saw 22
Skipping line 96168: expected 15 fields, saw 22
Skipping line 96866: expected 15 fields, saw 22
Skipping line 98175: expected 15 fields, saw 22
Skipping line 112539: expected 15 fields, saw 22
Skipping line 119377: expected 15 fields, saw 22
Skipping line 120065: expected 15 fields, saw 22
Skipping line 124703: expected 15 fields, saw 22

Skipping line 134024: expected 15 fields, saw 22
Skipping line 153938: expected 15 fields, saw 22
Skipping line 156225: expected 15 fields, saw 22
Skippin

# Converting gzip to TSV then make a balanced dataset and save in a CSV format

In [ ]:
# Load the dataset from a TSV file
file_path = "amazon_reviews_us_Office_Products_v1_00.tsv"  
df = pd.read_csv(file_path, sep='\t', error_bad_lines=False, dtype='str')

# Convert 'star_rating' column to integers after cleaning
df['star_rating'] = df['star_rating'].str.extract('(\d+)').astype(float).fillna(0).astype(int)

# Randomly shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Define the number of positive (high-rated) and negative (low-rated) reviews to include
sample_size = 1000000 // 2  # Balanced dataset with 50K positive and 500K negative reviews

# Select positive (4 and 5-star) and negative (1 and 2-star) reviews
positive_reviews = df[df['star_rating'] >= 4].sample(n=sample_size, random_state=42)
negative_reviews = df[df['star_rating'] <= 2].sample(n=sample_size, random_state=42)

# Concatenate positive and negative samples to create the balanced dataset
balanced_dataset = pd.concat([positive_reviews, negative_reviews], ignore_index=True)

# Shuffle the balanced dataset again (optional)
balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset to a new CSV file
balanced_dataset.to_csv("balanced_reviews.csv", index=False)

# Display the first few rows of the balanced dataset
print(balanced_dataset.head())

In [8]:
df = pd.read_csv('balanced_reviews.csv')

# Task 2 Part a & b

In [ ]:
# Gensim Model downloaded from the site of google.
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [4]:
pretrained_model = wv
amazon_reviews = df

In [5]:
X_train, X_test, y_train, y_test = train_test_split(amazon_reviews['review_body'], amazon_reviews['star_rating'], test_size=0.2, random_state=42)

In [6]:
X_train = X_train.astype(str)

sentences = [review.split() for review in X_train]

# Embedding size and window size
embedding_size = 300
window_size = 13
min_word_count = 9

# Train Word2Vec model
Word2vecModel = Word2Vec(sentences, vector_size=embedding_size, window=window_size, min_count=min_word_count)

# 2a

In [7]:
try:
    similarity3 = Word2vecModel.wv.similarity('king', 'queen')
    similarity4 = Word2vecModel.wv.similarity('excellent', 'outstanding')
    print(f"Semantic Similarity between 'king' and 'queen' ( Model): {similarity3}")
    print(f"Semantic Similarity between 'excellent' and 'outstanding' ( Model): {similarity4}")
except KeyError as e:
    print(f"Error: {e}")
    print("One or more words were not found in the Word2Vec model vocabulary.")


Error: "Key 'king' not present"
One or more words were not found in the Word2Vec model vocabulary.


# 2b

In [ ]:
# Example 1: King - Man + Woman = Queen
result1 = pretrained_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print(f"King - Man + Woman = {result1[0][0]}")

# Example 2: Excellent ~ Outstanding
result2 = pretrained_model.most_similar(positive=['excellent', 'outstanding'], topn=1)
print(f"Excellent ~ Outstanding = {result2[0][0]}")

# Example 3: Paris - France + Italy = Rome
result3 = pretrained_model.most_similar(positive=['paris', 'italy'], negative=['france'], topn=1)
print(f"Paris - France + Italy = {result3[0][0]}")

#3

In [9]:
X_test = X_test.fillna('')
tfidf_vectorizer = TfidfVectorizer(max_features=300)  # Adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [10]:

perceptron_w2v = Perceptron()
perceptron_w2v.fit(X_train_tfidf.toarray(), y_train)
y_pred_perceptron_w2v = perceptron_w2v.predict(X_test_tfidf.toarray())
accuracy_perceptron_w2v = accuracy_score(y_test, y_pred_perceptron_w2v)
print(f"Perceptron Accuracy (Word2Vec): {accuracy_perceptron_w2v}")

# SVM with Word2Vec features
svm_w2v = SVC()
svm_w2v.fit(X_train_tfidf.toarray(), y_train)
y_pred_svm_w2v = svm_w2v.predict(X_test_tfidf.toarray())
accuracy_svm_w2v = accuracy_score(y_test, y_pred_svm_w2v)
print(f"SVM Accuracy (Word2Vec): {accuracy_svm_w2v}")

Perceptron Accuracy (Word2Vec): 0.55
SVM Accuracy (Word2Vec): 0.6


#4 MLP

In [11]:
model = Word2Vec(sentences, vector_size=embedding_size, window=5, min_count=5)
embedding_size = 300
sentences = [review.split() for review in X_train]

def compute_avg_w2v_vectors(sentences, model, embedding_size):
    vectors = []
    for sentence in sentences:
        vector = [model.wv[word] for word in sentence if word in model.wv.index_to_key]
        if not vector:
            # If none of the words in the sentence are in the vocabulary, use a zero vector
            vectors.append(np.zeros(embedding_size))
        else:
            vectors.append(np.mean(vector, axis=0))
    return torch.tensor(vectors, dtype=torch.float32)

X_train_avg_w2v = compute_avg_w2v_vectors(sentences, model, embedding_size)
X_test_avg_w2v = compute_avg_w2v_vectors([review.split() for review in X_test], model, embedding_size)

C:\Users\Dell\AppData\Local\Temp\ipykernel_16368\1805663092.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  return torch.tensor(vectors, dtype=torch.float32)


In [12]:
# Check if a GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Define hyperparameters
input_size = embedding_size
hidden_size1 = 50
hidden_size2 = 5
num_classes = len(label_encoder.classes_)

# Move the model to the GPU
mlp_model = MLP(input_size, hidden_size1, hidden_size2, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)
batch_size = 64
num_epochs = 20

# Move the data to the GPU
X_train_avg_w2v = X_train_avg_w2v.to(device)
y_train_encoded = torch.tensor(y_train_encoded, dtype=torch.int64).to(device)
X_test_avg_w2v = X_test_avg_w2v.to(device)

train_dataset = TensorDataset(X_train_avg_w2v, y_train_encoded)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = mlp_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the MLP model on the testing split
with torch.no_grad():
    mlp_model.eval()
    outputs = mlp_model(X_test_avg_w2v)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == torch.tensor(y_test_encoded, dtype=torch.int64).to(device)).sum().item() / len(y_test_encoded)
    print(f'Accuracy on Testing Split: {accuracy * 100:.2f}%')

# Decode labels
decoded_predictions = label_encoder.inverse_transform(predicted.cpu().numpy())
print("Decoded Predictions:", decoded_predictions)

Epoch [1/20], Loss: 1.5346
Epoch [2/20], Loss: 1.5760
Epoch [3/20], Loss: 1.4238
Epoch [4/20], Loss: 1.4947
Epoch [5/20], Loss: 1.4399
Epoch [6/20], Loss: 1.4763
Epoch [7/20], Loss: 1.4334
Epoch [8/20], Loss: 1.5213
Epoch [9/20], Loss: 1.3632
Epoch [10/20], Loss: 1.4059
Epoch [11/20], Loss: 1.5216
Epoch [12/20], Loss: 1.4947
Epoch [13/20], Loss: 1.4611
Epoch [14/20], Loss: 1.4268
Epoch [15/20], Loss: 1.4320
Epoch [16/20], Loss: 1.4799
Epoch [17/20], Loss: 1.4367
Epoch [18/20], Loss: 1.3812
Epoch [19/20], Loss: 1.3838
Epoch [20/20], Loss: 1.4262
Accuracy on Testing Split: 40.00%
Decoded Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


# 5a RNN model

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the Simple RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

# Define hyperparameters
input_size = embedding_size
hidden_size = 10
num_classes = len(label_encoder.classes_)

# Move the model to the GPU
simple_rnn_model = SimpleRNN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_rnn_model.parameters(), lr=0.001)
batch_size = 64
num_epochs = 20

# Move the data to the GPU
X_train_avg_w2v = X_train_avg_w2v.to(device)
y_train_encoded = torch.tensor(y_train_encoded, dtype=torch.int64).to(device)
X_test_avg_w2v = X_test_avg_w2v.to(device)

train_dataset = TensorDataset(X_train_avg_w2v, y_train_encoded)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = simple_rnn_model(inputs.unsqueeze(1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the Simple RNN model on the testing split
with torch.no_grad():
    simple_rnn_model.eval()
    outputs = simple_rnn_model(X_test_avg_w2v.unsqueeze(1))
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == torch.tensor(y_test_encoded, dtype=torch.int64).to(device)).sum().item() / len(y_test_encoded)
    print(f'Accuracy on Testing Split (Simple RNN): {accuracy * 100:.2f}%')

Epoch [1/20], Loss: 1.2742
Epoch [2/20], Loss: 1.3480
Epoch [3/20], Loss: 1.3688
Epoch [4/20], Loss: 1.3209
Epoch [5/20], Loss: 1.3401
Epoch [6/20], Loss: 1.3370
Epoch [7/20], Loss: 1.3610
Epoch [8/20], Loss: 1.3203
Epoch [9/20], Loss: 1.3025
Epoch [10/20], Loss: 1.2617
Epoch [11/20], Loss: 1.2866
Epoch [12/20], Loss: 1.3816
Epoch [13/20], Loss: 1.2688
Epoch [14/20], Loss: 1.2414
Epoch [15/20], Loss: 1.2812
Epoch [16/20], Loss: 1.2844
Epoch [17/20], Loss: 1.2904
Epoch [18/20], Loss: 1.2775
Epoch [19/20], Loss: 1.2941
Epoch [20/20], Loss: 1.1929
Accuracy on Testing Split (Simple RNN): 40.00%


#5b GRU model

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the GRU RNN model
class GRURNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(GRURNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

# Define hyperparameters
input_size = embedding_size
hidden_size = 10
num_classes = len(label_encoder.classes_)

# Move the model to the GPU
gru_rnn_model = GRURNN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gru_rnn_model.parameters(), lr=0.001)
batch_size = 64
num_epochs = 50

# Move the data to the GPU
X_train_avg_w2v = X_train_avg_w2v.to(device)
y_train_encoded = torch.tensor(y_train_encoded, dtype=torch.int64).to(device)
X_test_avg_w2v = X_test_avg_w2v.to(device)

train_dataset = TensorDataset(X_train_avg_w2v, y_train_encoded)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = gru_rnn_model(inputs.unsqueeze(1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the GRU RNN model on the testing split
with torch.no_grad():
    gru_rnn_model.eval()
    outputs = gru_rnn_model(X_test_avg_w2v.unsqueeze(1))
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == torch.tensor(y_test_encoded, dtype=torch.int64).to(device)).sum().item() / len(y_test_encoded)
    print(f'Accuracy on Testing Split (GRU RNN): {accuracy * 100:.2f}%')

Epoch [1/50], Loss: 1.3505
Epoch [2/50], Loss: 1.3184
Epoch [3/50], Loss: 1.2855
Epoch [4/50], Loss: 1.3717
Epoch [5/50], Loss: 1.4305
Epoch [6/50], Loss: 1.3379
Epoch [7/50], Loss: 1.3707
Epoch [8/50], Loss: 1.3899
Epoch [9/50], Loss: 1.3965
Epoch [10/50], Loss: 1.3713
Epoch [11/50], Loss: 1.3546
Epoch [12/50], Loss: 1.4280
Epoch [13/50], Loss: 1.3559
Epoch [14/50], Loss: 1.3258
Epoch [15/50], Loss: 1.2483
Epoch [16/50], Loss: 1.2677
Epoch [17/50], Loss: 1.3434
Epoch [18/50], Loss: 1.3376
Epoch [19/50], Loss: 1.3580
Epoch [20/50], Loss: 1.3703
Epoch [21/50], Loss: 1.3562
Epoch [22/50], Loss: 1.3206
Epoch [23/50], Loss: 1.2983
Epoch [24/50], Loss: 1.2842
Epoch [25/50], Loss: 1.3310
Epoch [26/50], Loss: 1.2908
Epoch [27/50], Loss: 1.2744
Epoch [28/50], Loss: 1.3169
Epoch [29/50], Loss: 1.3577
Epoch [30/50], Loss: 1.3374
Epoch [31/50], Loss: 1.3163
Epoch [32/50], Loss: 1.2702
Epoch [33/50], Loss: 1.2534
Epoch [34/50], Loss: 1.3114
Epoch [35/50], Loss: 1.3092
Epoch [36/50], Loss: 1.3440
E

#5c LSTM model

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the LSTM model
class LSTMRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMRNN, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Define hyperparameters
input_size = embedding_size  # Input size matches the Word2Vec embedding size
hidden_size = 10
num_classes = len(label_encoder.classes_)

# Move the model to the GPU
lstm_rnn_model = LSTMRNN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_rnn_model.parameters(), lr=0.001)
batch_size = 64
num_epochs = 200

# Move the data to the GPU
X_train_avg_w2v = X_train_avg_w2v.to(device)
y_train_encoded = torch.tensor(y_train_encoded, dtype=torch.int64).to(device)
X_test_avg_w2v = X_test_avg_w2v.to(device)

train_dataset = TensorDataset(X_train_avg_w2v, y_train_encoded)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = lstm_rnn_model(inputs.unsqueeze(1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the LSTM RNN model on the testing split
with torch.no_grad():
    lstm_rnn_model.eval()
    outputs = lstm_rnn_model(X_test_avg_w2v.unsqueeze(1))
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == torch.tensor(y_test_encoded, dtype=torch.int64).to(device)).sum().item() / len(y_test_encoded)
    print(f'Accuracy on Testing Split (LSTM RNN): {accuracy * 100:.2f}%')

Epoch [1/200], Loss: 1.3657
Epoch [2/200], Loss: 1.3500
Epoch [3/200], Loss: 1.3614
Epoch [4/200], Loss: 1.3499
Epoch [5/200], Loss: 1.3669
Epoch [6/200], Loss: 1.3888
Epoch [7/200], Loss: 1.3627
Epoch [8/200], Loss: 1.3644
Epoch [9/200], Loss: 1.3411
Epoch [10/200], Loss: 1.3592
Epoch [11/200], Loss: 1.3560
Epoch [12/200], Loss: 1.3375
Epoch [13/200], Loss: 1.3439
Epoch [14/200], Loss: 1.3222
Epoch [15/200], Loss: 1.3280
Epoch [16/200], Loss: 1.3371
Epoch [17/200], Loss: 1.3406
Epoch [18/200], Loss: 1.3278
Epoch [19/200], Loss: 1.3240
Epoch [20/200], Loss: 1.3145
Epoch [21/200], Loss: 1.3047
Epoch [22/200], Loss: 1.3392
Epoch [23/200], Loss: 1.3261
Epoch [24/200], Loss: 1.2959
Epoch [25/200], Loss: 1.3337
Epoch [26/200], Loss: 1.3100
Epoch [27/200], Loss: 1.2906
Epoch [28/200], Loss: 1.2866
Epoch [29/200], Loss: 1.2988
Epoch [30/200], Loss: 1.2779
Epoch [31/200], Loss: 1.2715
Epoch [32/200], Loss: 1.3191
Epoch [33/200], Loss: 1.2709
Epoch [34/200], Loss: 1.2937
Epoch [35/200], Loss: 1